In [1]:
import torch
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

In [2]:
from torchvision import transforms, datasets
train_loader = torch.utils.data.DataLoader(
datasets.CIFAR10('./data/CIFAR_10/',
                train = True,
                download = True,
                 # Compose를 이용해 전처리를 진행.
                transform = transforms.Compose([
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    # 평균, 표준편차를 이용해 normalize
                    transforms.Normalize((0.5, 0.5, 0.5),
                                        (0.5, 0.5, 0.5))])), batch_size = 64, shuffle = True)

test_loader = torch.utils.data.DataLoader(
datasets.CIFAR10('./data/CIFAR_10/',
                train = False,
                transform = transforms.Compose([
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5),
                                        (0.5, 0.5, 0.5))])), batch_size = 64) # test는 shuffle이 필요 없음.

Files already downloaded and verified


In [12]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 8, kernel_size = 3, padding = 1)
        # self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 8, kernel_size = 3, padding = 1, stride = 2)
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 3, padding = 1)
        self.conv3 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, padding = 1)
        self.conv4 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1)

        # self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
        # self.fc1 = nn.Linear(8 * 8 * 16, 64)
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.fc1 = nn.Linear(2 * 2 * 64, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 10)
        
        # bn
        self.conv1_bn = nn.BatchNorm2d(8) # output chanel을 인자로.
        self.conv2_bn = nn.BatchNorm2d(16)
        self.conv3_bn = nn.BatchNorm2d(32)
        self.conv4_bn = nn.BatchNorm2d(64)
        
        # dropout
        self.dropout_p = 0.2
        
    def forward(self, x):
        x = self.conv1(x) # 32 * 32 * 3 -> 32 * 32 * 8 (kernel_size, padding, stride에 영향)
        x = self.conv1_bn(x) # bn 적용. 위치는 activation 전, 후. (명확히 밝혀지지 않음)
        '''
        activation은 비선형 함수이므로 앞의 내용과 많은 차이를 보임.
        보존되지 않은 데이터에 bn을 적용하는 것은 적합하지 않다는 의견.
        '''
        x = F.tanh(x)     # 32 * 32 * 8
        x = self.pool(x)  # 16 * 16 * 8 (pooling시 kernel_size, stride가 2이기 때문에 절반)
        
        x = self.conv2(x) # 16 * 16 * 8 -> 16 * 16 * 16
        x = self.conv2_bn(x)
        x = F.tanh(x)     # 16 * 16 * 16
        x = self.pool(x)  # 8 * 8 * 16
        
        x = self.conv3(x) # 8 * 8 * 16 -> 8 * 8 * 32
        x = self.conv3_bn(x)
        x = F.tanh(x)     # 8 * 8 * 32
        x = self.pool(x)  # 4 * 4 * 32
        
        x = self.conv4(x) # 4 * 4 * 32 -> 4 * 4 * 64
        x = self.conv4_bn(x)
        x = F.tanh(x)     # 4 * 4 * 64
        x = self.pool(x)  # 2 * 2 * 64. line 14, 37의 input.
        
        x = x.view(-1, 2 * 2 * 64)
        x = self.fc1(x)
        x = F.self.dropout(x, p = self.dropout_p) # dropout 역시 activation 전, 후.
        x = F.relu(x)
        x = self.fc2(x)
        x = F.self.dropout(x, p = self.dropout_p)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.log_softmax(x, dim = 1)
        return x
    
# Weight Initialize
import torch.nn.init as init
def weight_init(m):
    '''
    Ref: https://pytorch.org/docs/stable/nn.init.html
    
    균등분포) init.uniform_(tensorm, a = 0.0, b = 1.0) (a: Lower bound, b: Upper bound)
    정규분포) init.normal_(tensor, mean = 0.0, std = 1.0)
    init.xavier_uniform_(tensor, gain = 1.0)
    init.xavier_normal_(tensor, gain = 1.0)
    init.kaiming_uniform_(tensor, a = 0, mode = 'fan_in', nonlinearity = 'leakey_relu')
    init.kaiming_normal_(tensor, a = 0, mode = 'fan_in', nonlinearity = 'leakey_relu')
    '''
    
    if isinstance(m, nn.Conv2d):
        init.kaiming_uniform_(m.weight.data) # xavier_normal 분포에서 초기화
        if m.bias is not None:
            init.normal_(m.bias.data)
            
    elif isinstance(m, nn.BatchNorm2d):
        init.normal_(m.weight.data, mean = 1, std = 0.02)
        init.constant_(m.bias.data, 0)
        
    elif isinstance(m, nn.Linear):
        init.kaiming_uniform_(m.weight.data)
        init.normal_(m.bias.data)
    
    '''
    if isinstance(m, nn.Conv2d):
        init.uniform_(m.weight.data) # xavier_normal 분포에서 초기화
        if m.bias is not None:
            init.normal_(m.bias.data)
            
    elif isinstance(m, nn.BatchNorm2d):
        init.normal_(m.weight.data, mean = 1, std = 0.02)
        init.constant_(m.bias.data, 0)
        
    elif isinstance(m, nn.Linear):
        init.uniform_(m.weight.data)
        init.normal_(m.bias.data)
    '''
    
model = CNN().to(DEVICE)
model.apply(weight_init) # weight_init 적용
optimizer = optim.Adam(model.parameters(), lr = 0.001)
print('DEVICE: ', DEVICE)
print('MODEL: ', model)

DEVICE:  cpu
MODEL:  CNN(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=256, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
  (conv1_bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [13]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader): # train_loader 내 image와 label에 대해.
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{}({:.0f}%)]\tTrain Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item()))

In [14]:
def evaluate(model, test_loader):
    model.eval()
    
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction = 'sum').item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(target.view_as(prediction)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [8]:
EPOCHS = 3
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print('[{}] Test Loss: {:.4f}, accuracy: {:.2f}%\n'.format(epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/50000(0%)]	Train Loss: 4.650102
Train Epoch: 1 [6400/50000(13%)]	Train Loss: 2.310274
Train Epoch: 1 [12800/50000(26%)]	Train Loss: 2.074276
Train Epoch: 1 [19200/50000(38%)]	Train Loss: 1.914835
Train Epoch: 1 [25600/50000(51%)]	Train Loss: 1.667314
Train Epoch: 1 [32000/50000(64%)]	Train Loss: 1.516175
Train Epoch: 1 [38400/50000(77%)]	Train Loss: 1.613272
Train Epoch: 1 [44800/50000(90%)]	Train Loss: 1.556158
[1] Test Loss: 1.4879, accuracy: 46.00%

Train Epoch: 2 [0/50000(0%)]	Train Loss: 1.558417
Train Epoch: 2 [6400/50000(13%)]	Train Loss: 1.366343
Train Epoch: 2 [12800/50000(26%)]	Train Loss: 1.327870
Train Epoch: 2 [19200/50000(38%)]	Train Loss: 1.182071
Train Epoch: 2 [25600/50000(51%)]	Train Loss: 1.663728
Train Epoch: 2 [32000/50000(64%)]	Train Loss: 1.182510
Train Epoch: 2 [38400/50000(77%)]	Train Loss: 1.346214
Train Epoch: 2 [44800/50000(90%)]	Train Loss: 1.190817
[2] Test Loss: 1.2957, accuracy: 52.97%

Train Epoch: 3 [0/50000(0%)]	Train Loss: 1.174314


In [24]:
print("CNN's number of Parameters: ", sum([p.numel() for p in model.parameters()]))

CNN's number of Parameters:  43386


### 튜닝 과정 (아키텍처, 모델 구조 부분)

- 1. Activation tuning. (Parameter 수는 차이가 없음) (Parameter수가 model architecture의 크기)
    - sigmoid
    - tanh
    - leaky_relu
    
- 2. Convolution 층 증가. (각각의 층에서 파라미터 수의 변화를 계산할 수 있어야 함)
    - Parameter 수가 줄어듦
        - conv -> fc로 넘어갈 때 변화가 크게 일어남.
        - kernel_size와 stride가 2 => pooling size가 굉장히 큼.
            - feature map의 크기가 절반으로 계속 줄어들기 때문.

- 3. Pooling 교체.
    - MaxPooling
        - 최대값을 제외한 나머지 값은 고려되지 않음.
        - task) image를 input으로 받아 label로 분류. image에서 어느 특정 부분이 중요한지 focusing 하는 것이 중요.
    - AvgPooling
        - 모든 요소의 값을 고려.
        - task) pixel 하나하나가 중요할 경우. detecting 등.
        
- 4. stride
    - weight가 이동하는 간격.
    - stride가 1에서 2로 변경되면, 32 * 32 * 3 -> 16 * 16 * 8
    - parameter 수가 줄어듦.

### 튜닝 과정 (아키텍처 외적인 부분)
1. Weight initialization.

2. Batch Normalization.
    - Layer가 깊어짐에 따라 Gradient가 전달이 안될 때 사용.
        - ex) residual connection 방법
    - 깊이가 깊거나 parameter이 많을 때, train은 잘 맞추지만 test를 잘 맞추지 못할 경우,
        - 안정적 수렴.
        - 과적합 방지용.
            - 성능이 잘 나오는 모델에 추가로 적용하는 것을 추천. (90% 이상)
            
3. Dropout
    - 학습 때마다 노드의 Weight를 랜덤으로 초기화.
        - 앙상블 효과가 있음.
        - 성능이 잘 나오는 모델에 추가로 적용하는 것을 추천.
        
4. Batch Size
    - 64개의 데이터를 넣은 결과 64개의 loss를 평균 내서 역전파 진행.
    - Batch Size가 커지면 iteration이 작아짐.
        - ex) Batch Size = 1이면 데이터 한 개 마다 역전파를 진행. 학습을 많이 진행한다는 의미. (느림)
    - Batch Normalization과 연관.
    - 2의 배수 승으로 초기화. 32, 64, 128, 256, 512 등
    
5. Optimizer
    - optim. tab으로 확인.
    - lr 등 하이퍼 파라미터도 조정.